# BATTLE GROUND DATA SET

<h2> 1. 데이터 셋 확인하기

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import keras
import sklearn
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df_train = pd.read_csv('train_V2.csv')
df_test = pd.read_csv('test_V2.csv')

In [ ]:
df_train.shape, df_test.shape

각 파일을 확인해보니<br>
train 할 데이터의 수는 4,446,966 개,<br>
test 할 데이터의 수는 1,934,175개 입니다.<br>

In [ ]:
df_train.columns

각각의 attribute 가 있고, 예측해야할 것은 'winPlacePerc

In [ ]:
df_test.columns

* Id - 플레이어의 Id
* groupId - 경기 내의 그룹을 식별하는 ID. 현재 그룹의 선수들이 서로 다른 경기에서 경기한다면, 그들은 매번 다른 groupId를 갖게 될 것이다.
* matchId - 경기를 식별하기 위한 ID. train set과 test set에 모두 있는 시합은 없다.
* matchDuration - 경기 시간.
* matchType - 경기 종류(duo, quad ...).
* assists - 팀 동료들과 같이 죽인 적 수이다.
* boosts - 부스트 아이템 사용한 수.
* damageDealt - 가한 데미지 총량. Note: 자신에게 가한 데미지는 제외.
* DBNOs - 빈사상태로 만든 적의 수.
* headshotKills - 헤드샷 으로 처리한 적의 수.
* heals - 치료 아이템 사용 수.
* killPlace - 경기에서 처치한 적의 수 랭킹.
* killPoints - 플레이어의 처치 기반 외부 랭킹 (Elo 방식의 순위). rankPoints에서 -1이 아닌 값이 있는 경우, killPoints에서 0은 "없음"으로 처리되어야 한다.
* kills - 처치한 적의 수.
* killStreaks - 단기간에 가장 많이 처치한 적의 최대치.
* longestKill - 플레이어가 적을 죽인 가장 긴 거리. 선수를 다운시키고 멀리 운전하는 것이 가장 긴 처치로 이어질 수 있기 때문에 오해의 소지가 있을 수 있다.
* maxPlace - 경기에서 가장 순위가 낮은 것에 대한 데이터. 이것은 순위를 건너뛸 수도 있기 때문에 numGroups와 일치하지 않을 수 있다.
* numGroups - 경기에 있는 팀의 수.
* rankPoints - Elo 방식의 플레이어 랭킹. 다음 버전의 API에서는 삭제될 예정이기 때문에 사용에 주의. '-1' 값은 순위가 "None" 이다.
* revives - 플레이어가 팀원 회복 시킨 수.
* rideDistance - 차량으로 이동한 거리(단위 : 미터).
* roadKills - 차량으로 죽인 플레이어 수.
* swimDistance - 수영한 거리(단위 : 미터).
* teamKills - 팀킬한 횟수.
* vehicleDestroys - 차량을 폭파시킨 횟수.
* walkDistance - 걸은 총 거리(단위 : 미터).
* weaponsAcquired - 무기 얻은 갯수.
* winPoints - 플레이어의 승리 기반 외부 랭킹 (Elo 방식의 순위). rankPoints에서 -1이 아닌 값이 있는 경우, winPoints에서 0은 "없음"으로 처리되어야 한다.
* winPlacePerc - 예측 목표. 순위의 퍼센트로 표시되며, 1이면 경기에서 1등이고 0이면 경기에서 꼴지 했다는 것이다. 이것은 numGroups로 계산되는게 아니라, maxPlace로 계산되기 때문에 누락되는 것이 있을수도다.

파일을 출력해보면

In [ ]:
df_train.head()

각 feature를 살펴보면

In [ ]:
df_train.dtypes

In [ ]:
df_train.describe()

In [ ]:
df_train.isnull().sum()

In [ ]:
df_test.isnull().sum()

train 데이터에서 winPlacePerc 피쳐에 1개의 결측치가 있당

<h2> 2. 탐색적 데이터 분석 (EDA, Exploratory Data Analysis)

<h3> 2-1. assists

In [ ]:
df_copy = df_train.copy()

In [ ]:
df_train['assists'].mean()

In [ ]:
df_train['assists'].quantile(0.99)

In [ ]:
df_train['assists'].max()

In [ ]:
df_copy1 = df_train.copy()
df_copy1.loc[df_copy1['assists'] > df_copy1['assists'].quantile(0.99)] = '4+'
sns.countplot(df_copy1['assists'].astype('str').sort_values())
plt.title("Count of Assistant",fontsize=15)

대부분의 사람들이 어시스트를 하지 않는다

In [ ]:
df_copy2 = df_train.copy()
df_copy2 = df_copy2[df_copy2['assists']==0]
plt.figure(figsize=(15, 10))
plt.title("Kill Count of 0 Assists",fontsize=15)
sns.distplot(df_copy2['kills'])

어이스트가 없는 플레이어는 킬 또한 하지 못한다 ㅠㅠ

<h2> 2-2. Heal & Boosts


In [ ]:
df_train['boosts'].mean(), df_train['heals'].mean()

In [ ]:
df_train['boosts'].quantile(0.99), df_train['heals'].quantile(0.99)

In [ ]:
df_train['boosts'].max(), df_train['heals'].max()

In [ ]:
df_copy3 = df_train.copy()
df_copy3.loc[df_copy3['boosts'] > df_copy3['boosts'].quantile(0.99)] = '8+'
sns.countplot(df_copy3['boosts'].astype('str').sort_values())
plt.title("Count of Boosts used",fontsize=15)

In [ ]:
df_copy4 = df_train.copy()
df_copy4.loc[df_copy4['heals'] > df_copy4['heals'].quantile(0.99)] = '13+'
sns.countplot(df_copy4['heals'].astype('str'))
plt.title("Count of Heal used",fontsize=15)

대부분의 사람들이 boost 아이템과 heal 아이템을 사용하지 않는다<br>
이러한 아이템은 당연히 우승과 어떤 관련이 있을까?(당연 많이쓸수록 우승확률이 높아지겠찌만)

In [ ]:
plt.figure(figsize=(8, 6))
sns.boxplot(x='boosts', y="winPlacePerc", data=df_train)

In [ ]:
plt.figure(figsize=(8, 6))
sns.boxplot(x='heals', y="winPlacePerc", data=df_train)

In [ ]:
sns.jointplot(x="winPlacePerc", y="boosts", data=df_train)

In [ ]:
sns.jointplot(x="winPlacePerc", y="heals", data=df_train)

이로써 부스트, 힐링 아이템은 우승과 양의 상관관계를 가진다!

<h2> 2-3. Kill

In [ ]:
df_train['kills'].mean()

In [ ]:
df_train['kills'].quantile(0.99)

In [ ]:
df_train['kills'].max()

In [ ]:
df_copy = df_train.copy()
df_copy.loc[df_copy['kills'] > 7 ] = '8+'
sns.countplot(df_copy['kills'].astype(str).sort_values())
plt.title('Count of Kill')

대부분의 플레이어가 1킬도 못하고 죽는당 ㅠㅠ

In [ ]:
df_copy = df_train.copy()
df_copy = df_copy[df_copy['kills'] > 0]
plt.figure(figsize=(15, 10))
plt.title('headshot by 0 killers',fontsize=15)
sns.distplot(df_train['headshotKills'])

또한 보통 플레이어가 헤드샷을 하지 않고(못하고), 1에서 2 해드샷 킬 정도 한다

In [ ]:
df_copy = df_train.copy()
df_copy = df_copy[df_copy['kills'] == 0]
plt.figure(figsize=(15, 10))
plt.title('damage dealt for 0 kill players',fontsize=15)
sns.distplot(df_train['damageDealt'])

0킬 한 플레이어들은 또한 다른 플레이어들에게 데미지를 주지 못한다..

In [ ]:
sns.jointplot(x="winPlacePerc", y="kills", data=df_train)

킬 수 또한 우승확률과 양의 상관관계가 이따

In [ ]:
sns.countplot(df_train['teamKills'])

대부분의 사람들이 팀킬은 하지 않는다

In [ ]:
sns.jointplot(x="winPlacePerc", y="teamKills", data=df_train)

<h2> 2-4. Revive

In [ ]:
df_train['revives'].value_counts()

In [ ]:
df_train['revives'].quantile(0.99)

In [ ]:
df_train[df_train['revives'] > 3]['revives'].sum()

In [ ]:
labels = ['0', '1', '2', '3', '3+']
sizes = [3859867, 470110, 95545, 17333, 19034]
colors = ['yellowgreen', 'gold', 'lightskyblue', 'lightcoral', 'black']
explode = (1, 1, 1, 1, 1)
plt.title("Perc. of Revives", fontsize=15)
plt.pie(sizes, colors=colors, autopct='%1.1f%%', labels=labels)
plt.axis('equal')

약 86.5% 플레이어가 부활하지 못한다

<h2> 2-5. Distance

In [ ]:
plt.figure(figsize=(15, 10))
sns.distplot(df_train['rideDistance'], color='red')
plt.title('Ride Distance', fontsize=15)

In [ ]:
plt.figure(figsize=(15, 10))
sns.distplot(df_train['swimDistance'], color='blue')
plt.title('Swim Distance', fontsize=15)

In [ ]:
plt.figure(figsize=(15, 10))
sns.distplot(df_train['walkDistance'], color='green')
plt.title('Walk Distance', fontsize=15)

In [ ]:
sns.jointplot(x="rideDistance", y="kills", data=df_train)

차를 타고 가는 거리가 멀어질 수록 킬 수는 적어진다

In [ ]:
sns.jointplot(x="swimDistance", y="kills", data=df_train)

In [ ]:
sns.jointplot(x="walkDistance", y="kills", data=df_train)

거리는 길어질 수록 킬 수가 적어진다

In [ ]:
sns.jointplot(x="winPlacePerc", y="walkDistance", data=df_train)

하지만 거리가 길어질 수록 승리 확률은 높아진다

<h2> 2-6. Others

In [ ]:
df_train['vehicleDestroys'].value_counts()

In [ ]:
df_train[df_train['vehicleDestroys']>1]['vehicleDestroys'].sum()

In [ ]:
labels = ['0', '1', '2+']
sizes = [4413212, 32422, 2790]
colors = ['yellowgreen', 'gold', 'lightskyblue']
explode = (0, 0, 0)
plt.title("Perc. of Vehicle Destroys", fontsize=15)
plt.pie(sizes, colors=colors, autopct='%1.1f%%', labels=labels)
plt.axis('equal')

In [ ]:
plt.scatter(x="weaponsAcquired",y="winPlacePerc", data=df_train)

<h2> 2-7. Corr.

In [ ]:
f,ax = plt.subplots(figsize=(15, 15))
sns.heatmap(df_train.corr(), annot=True, linewidths=.5, fmt= '.1f')
plt.show()

In [ ]:
k = 10
corr = df_train.corr()
column = corr.nlargest(k, 'winPlacePerc').index

In [ ]:
colcoef = np.corrcoef(df_train[column].values.T)

In [ ]:
f, ax = plt.subplots(figsize=(15, 10))
sns.heatmap(colcoef, annot=True, xticklabels=column.values, yticklabels=column.values)

In [ ]:
df_train[df_train['winPlacePerc'].isnull()]

In [ ]:
df_train[df_train.Id == 'f70c74418bb064']

In [ ]:
df_train = df_train[df_train.Id != 'f70c74418bb064']

In [ ]:
df_train

In [ ]:
df_train.head()

In [ ]:
df_train["distance"] = df_train["rideDistance"] + df_train["walkDistance"] + df_train["swimDistance"]
df_test["distance"] = df_test["rideDistance"] + df_test["walkDistance"] + df_test["swimDistance"]
df_train.drop(['rideDistance','walkDistance','swimDistance','headshotKills','roadKills'],inplace=True, axis=1)
df_test.drop(['rideDistance','walkDistance','swimDistance','headshotKills','roadKills'],inplace=True,axis=1)

In [ ]:
df_train.head()

In [ ]:
df_train["items"] = df_train["boosts"] + df_train["heals"]
df_test["items"] = df_train["boosts"] + df_train["heals"]
df_train.drop(['boosts', 'heals'],inplace=True, axis=1)
df_test.drop(['boosts', 'heals'],inplace=True, axis=1)

In [ ]:
k = 12
corr = df_train.corr()
column = corr.nlargest(k, 'winPlacePerc').index
colcoef = np.corrcoef(df_train[column].values.T)
f, ax = plt.subplots(figsize=(15, 10))
sns.heatmap(colcoef, annot=True, xticklabels=column.values, yticklabels=column.values)

In [ ]:
f,ax = plt.subplots(figsize=(15, 15))
sns.heatmap(df_train.corr(), annot=True, linewidths=.5, fmt= '.1f')
plt.show()

In [ ]:
df_train.columns

In [ ]:
train_feat = ['assists', 'damageDealt', 'DBNOs',
       'killPlace', 'killPoints', 'kills', 'killStreaks', 'longestKill',
       'maxPlace', 'numGroups', 'rankPoints',
       'revives', 'vehicleDestroys', 'weaponsAcquired',
       'winPoints', 'distance', 'items']

In [ ]:
X = df_train[train_feat]

In [ ]:
Y = df_train['winPlacePerc']

In [ ]:
X.shape

In [ ]:
Y.shape

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=133)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
MMS = MinMaxScaler(feature_range=(0, 1))

X_scal = MMS.fit_transform(X_train)
X_train = pd.DataFrame(X_scal)

X_scal = MMS.transform(X_test)
X_test = pd.DataFrame(X_scal)

In [ ]:
from sklearn.linear_model import LinearRegression
# from sklearn.metrics import mean_absolute_error

# Instantiate
linreg = LinearRegression()

# Fit
linreg.fit(X_train, y_train)

# Predict
y_pred = linreg.predict(X_test)

# Evaluate
# mean_absolute_error(y_pred, Y_test)

In [ ]:
y_pred

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error

# Instantiate the classifier
tree = DecisionTreeRegressor()

# Train model on training set
tree.fit(X_train, y_train)

# Predict
y_pred = tree.predict(X_test)